In [2]:
!pip install transformers

     |████████████████████████████████| 890kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 23.3MB/s 
     |████████████████████████████████| 3.0MB 31.7MB/s 
     |████████████████████████████████| 890kB 46.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=37f5442434c6e3d1b6fc2b9f00a388b2e592ff0cfdc761b60b5ff79ca29677d9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
def get_data():
    ! wget https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
    ! mkdir data
    ! unzip -q cola_public_1.1.zip -d ./data
get_data()

--2020-09-12 19:31:16--  https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
Resolving nyu-mll.github.io (nyu-mll.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to nyu-mll.github.io (nyu-mll.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255330 (249K) [application/zip]
Saving to: ‘cola_public_1.1.zip’

cola_public_1.1.zip 100%[===================>] 249.35K  --.-KB/s    in 0.03s   

2020-09-12 19:31:17 (7.22 MB/s) - ‘cola_public_1.1.zip’ saved [255330/255330]



In [6]:
df = pd.read_csv('./data/cola_public/raw/in_domain_train.tsv', delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [7]:
class GLUE(Dataset):
    def __init__(self, data, tokenizer, max_len):
        super().__init__()
        self.X = data.sentence.values
        self.y = data.label.values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        inputs = self.tokenizer.tokenize(self.X[idx])
        inputs = self.tokenizer.encode_plus(inputs,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                            truncation=True)
        
        return torch.LongTensor(inputs['input_ids']), torch.LongTensor(inputs['attention_mask']), self.y[idx]


In [57]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 4
LEARNING_RATE = 3e-5
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# testing the dataset
test = GLUE(df.head(), tokenizer, MAX_LEN)
test[0]

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(tensor([  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
          2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [10]:
# splitting up the data and throwing it in a dataloader
from sklearn.model_selection import train_test_split

X_train, X_test, _, _ = train_test_split(df, df.label, test_size=.2, random_state=42)
train_ds = GLUE(X_train, tokenizer, MAX_LEN)
valid_ds = GLUE(X_test, tokenizer, MAX_LEN)

train_dl = DataLoader(train_ds, shuffle=True, batch_size=TRAIN_BATCH_SIZE)
valid_dl = DataLoader(valid_ds, batch_size=VALID_BATCH_SIZE)

In [11]:
x, mask, y = next(iter(train_dl))
x

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([[  101,  1996,  9540,  ...,     0,     0,     0],
        [  101,  5742,  2003,  ...,     0,     0,     0],
        [  101,  2952,  1051,  ...,     0,     0,     0],
        ...,
        [  101,  2984, 16849,  ...,     0,     0,     0],
        [  101, 20328, 17645,  ...,     0,     0,     0],
        [  101,  1045,  2404,  ...,     0,     0,     0]])

In [12]:
x.size(), mask.size(), y.size()

(torch.Size([16, 128]), torch.Size([16, 128]), torch.Size([16]))

In [13]:
class BERT(nn.Module):
    def __init__(self, dropout):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.classifier = torch.nn.Linear(768, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        seq, pool = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = F.relu(self.pre_classifier(pool))
        output = self.dropout(output)
        return self.classifier(output)

In [14]:
model = BERT(.3)
model.to(device)

BERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [72]:
def train_model(model, train_dl, valid_dl, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss, total, n_correct = 0, 0, 0
        
        for idx, data in enumerate(train_dl):
            ids = data[0].to(device)
            mask = data[1].to(device)
            targets = torch.LongTensor(data[2]).to(device)
            batch_size = targets.shape[0]

            y_pred = model(ids, mask)
            loss = F.binary_cross_entropy_with_logits(y_pred, targets.float().unsqueeze(1))
            total_loss += loss.item() * batch_size
            total += batch_size
            out = (y_pred >= 0.5).float().squeeze()
            n_correct += out.eq(targets).sum().item()

            if idx % 100 == 0:
                print(f"Training Loss per 100 steps: {total_loss / total}")
                print(f"Training Accuracy per 100 steps: {n_correct/total}")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        val_loss, val_acc = valid_model(model, valid_dl)
        print(f"Epoch {epoch+1} Training Loss: {total_loss/total} Training Accuracy: {n_correct/total} Valid Loss: {val_loss} Valid Accuracy: {val_acc}")    

In [73]:
def valid_model(model, valid_dl):
    model.eval()
    total_loss, total, n_correct = 0, 0, 0
    for data in valid_dl:
        ids = data[0].to(device)
        mask = data[1].to(device)
        targets = torch.LongTensor(data[2]).to(device)
        batch_size = targets.shape[0]

        y_pred = model(ids, mask)
        loss = F.binary_cross_entropy(torch.sigmoid(y_pred), targets.float().unsqueeze(1))
        total_loss += loss.item() * batch_size
        total += batch_size
        out = (y_pred >= 0.5).float().squeeze()
        n_correct += out.eq(targets).sum().item()

    return total_loss/total, n_correct/total

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train_model(model, train_dl, valid_dl, optimizer, EPOCHS)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 100 steps: 0.0057329232804477215
Training Accuracy per 100 steps: 1.0
Training Loss per 100 steps: 0.058381374739802046
Training Accuracy per 100 steps: 0.9832920792079208
Training Loss per 100 steps: 0.07066349477671661
Training Accuracy per 100 steps: 0.9791666666666666
Training Loss per 100 steps: 0.07334175968637065
Training Accuracy per 100 steps: 0.9775747508305648
Training Loss per 100 steps: 0.07907062089401634
Training Accuracy per 100 steps: 0.9738154613466334
Epoch 1 Training Loss: 0.07811889574394631 Training Accuracy: 0.974561403508772 Valid Loss: 0.7202749164575795 Valid Accuracy: 0.8158971361776739
Training Loss per 100 steps: 0.017747756093740463
Training Accuracy per 100 steps: 1.0
Training Loss per 100 steps: 0.056130996077681086
Training Accuracy per 100 steps: 0.9746287128712872
Training Loss per 100 steps: 0.06342475089552668
Training Accuracy per 100 steps: 0.9760572139303483
Training Loss per 100 steps: 0.064611841165275
Training Accuracy per 10